### **5 - Encodings (Codificaciones)**

#### **Introducción**

Cualquier documento HTML o XML está escrito en una codificación específica como `ASCII` o `UTF-8`. Pero cuando cargues ese documento en Beautiful Soup, descubrirás que ha sido convertido a Unicode:

In [7]:
from bs4 import BeautifulSoup

markup = "<h1>Sacr\xc3\xa9 bleu!</h1>"
soup = BeautifulSoup(markup)
soup.h1

<h1>SacrÃ© bleu!</h1>

In [2]:
soup.h1.string

'SacrÃ© bleu!'

No es magia. Beautiful Soup utiliza una sub-librería llamada `Unicode, Dammit` para detectar la codificación de un documento y convertirlo a Unicode. La codificación autodetectada está disponible como atributo `.original_encoding` del objeto BeautifulSoup:

In [10]:
soup.original_encoding  # 'utf-8'

`Unicode, Dammit` adivina correctamente la mayoría de las veces, pero a veces se equivoca. A veces adivina correctamente, pero sólo después de una búsqueda byte a byte del documento que lleva mucho tiempo. Si conoces la codificación de un documento de antemano, puedes evitar errores y retrasos pasándosela al constructor de BeautifulSoup como from_encoding.

Aquí hay un documento escrito en ISO-8859-8. El documento es tan corto que Unicode, Dammit no puede fijarse en él, y lo identifica erróneamente como ISO-8859-7:

In [13]:
markup = b"<h1>\xed\xe5\xec\xf9</h1>"
soup = BeautifulSoup(markup)
soup.h1

<h1>翴檛</h1>

In [14]:
soup.original_encoding

'Big5'

Podemos solucionarlo introduciendo la codificación `from_encoding`:

In [15]:
soup = BeautifulSoup(markup, from_encoding="iso-8859-8")
soup.h1

<h1>םולש</h1>

In [16]:
soup.original_encoding

'iso-8859-8'

Si no sabes cuál es la codificación correcta, pero sabes que `Unicode, Dammit` está adivinando mal, puedes pasar las conjeturas erróneas como `exclude_encodings`:

In [17]:
soup = BeautifulSoup(markup, exclude_encodings=["ISO-8859-7"])
soup.h1

<h1>翴檛</h1>

In [18]:
soup.original_encoding  # 'WINDOWS-1255'

'Big5'

Windows-1255 no es 100% correcto, pero esa codificación es un superconjunto compatible de ISO-8859-8, así que se acerca bastante. (exclude_encodings es una nueva función de Beautiful Soup 4.4.0.)

En raras ocasiones (normalmente cuando un documento UTF-8 contiene texto escrito en una codificación completamente diferente), la única forma de obtener Unicode puede ser reemplazar algunos caracteres con el carácter especial Unicode "REPLACEMENT CHARACTER" (U+FFFD, �). Si Unicode, Dammit necesita hacer esto, establecerá el atributo `.contains_replacement_characters` a `True` en el objeto UnicodeDammit o BeautifulSoup. Esto le permite saber que la representación Unicode no es una representación exacta del original -algunos datos se perdieron. Si un documento contiene �, pero `.contains_replacement_characters` es `False`, sabrá que la � estaba ahí originalmente (como en este párrafo) y no sustituye a los datos que faltan.

#### **Codificación de salida**

Cuando escribes un documento desde Beautiful Soup, obtienes un documento `UTF-8`, incluso si el documento no estaba en `UTF-8` al principio. Aquí hay un documento escrito en la codificación `Latin-1`:

In [20]:
markup = b'''
 <html>
  <head>
   <meta content="text/html; charset=ISO-Latin-1" http-equiv="Content-type" />
  </head>
  <body>
   <p>Sacr\xe9 bleu!</p>
  </body>
 </html>
'''

soup = BeautifulSoup(markup)
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
 </head>
 <body>
  <p>
   Sacré bleu!
  </p>
 </body>
</html>



Tenga en cuenta que la etiqueta `<meta>` se ha reescrito para reflejar el hecho de que el documento está ahora en `UTF-8`.

Si no desea `UTF-8`, puede pasar una codificación a `prettify()`:

In [21]:
print(soup.prettify("latin-1"))

b'<html>\n <head>\n  <meta content="text/html; charset=latin-1" http-equiv="Content-type"/>\n </head>\n <body>\n  <p>\n   Sacr\xe9 bleu!\n  </p>\n </body>\n</html>\n'


También puedes llamar a `encode()` sobre el objeto BeautifulSoup, o sobre cualquier elemento de soup, como si fuera una string de Python:

In [22]:
soup.p.encode("latin-1")

b'<p>Sacr\xe9 bleu!</p>'

In [23]:
soup.p.encode("utf-8")

b'<p>Sacr\xc3\xa9 bleu!</p>'

Los caracteres que no puedan representarse en la codificación elegida se convertirán en referencias numéricas a entidades XML. A continuación se muestra un documento que incluye el carácter Unicode SNOWMAN:

In [24]:
markup = u"<b>\N{SNOWMAN}</b>"
snowman_soup = BeautifulSoup(markup)
tag = snowman_soup.b

El carácter SNOWMAN puede formar parte de un documento UTF-8 (se parece a ☃), pero no existe representación para ese carácter en ISO-Latin-1 o ASCII, por lo que se convierte en "&#9731" para esas codificaciones:

In [25]:
print(tag.encode("utf-8"))  # <b>☃</b>

b'<b>\xe2\x98\x83</b>'


In [27]:
print (tag.encode("latin-1"))

b'<b>&#9731;</b>'


In [28]:
print (tag.encode("ascii"))

b'<b>&#9731;</b>'


#### **Unicode, Dammit**

Puedes usar `Unicode, Dammit` sin usar Beautiful Soup. Es útil cuando tienes datos en una codificación desconocida y sólo quieres que se conviertan en Unicode:

In [29]:
from bs4 import UnicodeDammit

dammit = UnicodeDammit("Sacr\xc3\xa9 bleu!")
print(dammit.unicode_markup)

SacrÃ© bleu!


In [30]:
dammit.original_encoding  # 'utf-8'

Las suposiciones de `Unicode, Dammit` serán mucho más precisas si instalas las bibliotecas `chardet` o `cchardet` de Python. Cuantos más datos le proporciones a `Unicode, Dammit`, más acertadas serán sus suposiciones. Si tienes tus propias sospechas sobre cuál podría ser la codificación, puedes pasarlas como una lista:

In [31]:
pip install chardet

  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   ------ -------------------------------- 30.7/199.4 kB 262.6 kB/s eta 0:00:01
   -------- ------------------------------ 41.0/199.4 kB 281.8 kB/s eta 0:00:01
   ------------ -------------------------- 61.4/199.4 kB 297.7 kB/s eta 0:00:01
   ------------------------------- ------ 163.8/199.4 kB 701.4 kB/s eta 0:00:01
   -------------------------------------- 199.4/199.4 kB 713.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
pip install cchardet

  Using cached cchardet-2.1.7.tar.gz (653 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build cchardet
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for cchardet (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [11 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\cchardet
      copying src\cchardet\version.py -> build\lib.win-amd64-cpython-312\cchardet
      copying src\cchardet\__init__.py -> build\lib.win-amd64-cpython-312\cchardet
      running build_ext
      building 'cchardet._cchardet' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cchardet
ERROR: Could not build wheels for cchardet, which is required to install pyproject.toml-based

In [34]:
dammit = UnicodeDammit("Sacr\xe9 bleu!", "latin-1", "iso-8859-1")
print(dammit.unicode_markup)

Sacré bleu!


In [35]:
dammit.original_encoding  # 'latin-1'

##### **Codificaciones incoherentes**

A veces un documento está mayoritariamente en UTF-8, pero contiene caracteres de Windows-1252 como (de nuevo) las smart quotes de Microsoft. Esto puede ocurrir cuando un sitio web incluye datos de múltiples fuentes. Puede utilizar `UnicodeDammit.detwingle()` para convertir un documento de este tipo en UTF-8 puro. He aquí un ejemplo sencillo:

In [36]:
snowmen = (u"\N{SNOWMAN}" * 3)
quote = (u"\N{LEFT DOUBLE QUOTATION MARK}I like snowmen!\N{RIGHT DOUBLE QUOTATION MARK}")
doc = snowmen.encode("utf8") + quote.encode("windows_1252")

Este documento es un desastre. Los muñecos de nieve están en UTF-8 y las quotes en Windows-1252. Puedes mostrar los muñecos de nieve o las comillas (quotes), pero no ambos:

In [37]:
print(doc)  # # ☃☃☃�I like snowmen!�

b'\xe2\x98\x83\xe2\x98\x83\xe2\x98\x83\x93I like snowmen!\x94'


In [38]:
print(doc.decode("windows-1252"))

â˜ƒâ˜ƒâ˜ƒ“I like snowmen!”


Decodificar el documento como UTF-8 genera un error UnicodeDecodeError, y decodificarlo como Windows-1252 te da un galimatías. Afortunadamente, `UnicodeDammit.detwingle()` convertirá el string a UTF-8 puro, permitiéndole decodificarlo a Unicode y mostrar los muñecos de nieve y las comillas simultáneamente:

In [39]:
new_doc = UnicodeDammit.detwingle(doc)
print(new_doc.decode("utf8"))

☃☃☃“I like snowmen!”


`UnicodeDammit.detwingle()` sólo sabe cómo manejar Windows-1252 incrustado en UTF-8 (o viceversa, supongo), pero este es el caso más común.

Tenga en cuenta que debe saber llamar a `UnicodeDammit.detwingle()` en sus datos antes de pasarlos a BeautifulSoup o al constructor UnicodeDammit. Beautiful Soup asume que un documento tiene una única codificación, sea cual sea. Si le pasas un documento que contiene tanto UTF-8 como Windows-1252, es probable que piense que todo el documento es Windows-1252, y el documento saldrá con un aspecto como el de â˜ƒâ˜ƒâ˜ƒ“I like snowmen!”.